In [4]:
# Dependently Typed Language Interpreter (Corrected Version)

from dataclasses import dataclass
from typing import Any, Dict

# === Syntax ===

@dataclass(frozen=True)
class Term:
    pass

@dataclass(frozen=True)
class Var(Term):
    name: str
    def __repr__(self):
        return self.name

@dataclass(frozen=True)
class Type(Term):
    def __repr__(self):
        return "Type"

@dataclass(frozen=True)
class Pi(Term):
    var: str
    var_type: Term
    body: Term
    def __repr__(self):
        return f"({self.var}:{self.var_type}) -> {self.body}"

@dataclass(frozen=True)
class Lambda(Term):
    var: str
    var_type: Term
    body: Term
    def __repr__(self):
        return f"({self.var}:{self.var_type}. {self.body})"

@dataclass(frozen=True)
class App(Term):
    func: Term
    arg: Term
    def __repr__(self):
        return f"({self.func} {self.arg})"

# === Simple Type Environment ===

class Env(Dict[str, Term]):
    def extend(self, x: str, t: Term):
        new_env = Env(self)
        new_env[x] = t
        return new_env

# === Type Checker ===

def type_check(env: Env, term: Term) -> Term:
    if isinstance(term, Type):
        return Type()
    elif isinstance(term, Var):
        if term.name in env:
            return env[term.name]
        else:
            raise TypeError(f"Unbound variable {term.name}")
    elif isinstance(term, Pi):
        t_type = type_check(env, term.var_type)
        if t_type != Type():
            raise TypeError("Pi domain must be a type")
        new_env = env.extend(term.var, term.var_type)
        body_type = type_check(new_env, term.body)
        if body_type != Type():
            raise TypeError("Pi codomain must be a type")
        return Type()
    elif isinstance(term, Lambda):
        t_type = type_check(env, term.var_type)
        if t_type != Type():
            raise TypeError("Lambda parameter type must be a Type")
        new_env = env.extend(term.var, term.var_type)
        body_type = type_check(new_env, term.body)
        return Pi(term.var, term.var_type, body_type)
    elif isinstance(term, App):
        func_type = type_check(env, term.func)
        if not isinstance(func_type, Pi):
            raise TypeError(f"Cannot apply non-function {func_type}")
        arg_type = type_check(env, term.arg)
        if arg_type != func_type.var_type:
            raise TypeError(f"Argument type mismatch: expected {func_type.var_type}, got {arg_type}")
        return substitute(func_type.body, func_type.var, term.arg)
    else:
        raise TypeError(f"Unknown term {term}")

# === Substitution ===

def substitute(term: Term, var: str, val: Term) -> Term:
    if isinstance(term, Var):
        return val if term.name == var else term
    elif isinstance(term, Lambda):
        if term.var == var:
            return term
        return Lambda(term.var, substitute(term.var_type, var, val), substitute(term.body, var, val))
    elif isinstance(term, App):
        return App(substitute(term.func, var, val), substitute(term.arg, var, val))
    elif isinstance(term, Pi):
        if term.var == var:
            return term
        return Pi(term.var, substitute(term.var_type, var, val), substitute(term.body, var, val))
    else:
        return term

# === Normalization (Beta Reduction) ===

def normalize(term: Term) -> Term:
    if isinstance(term, App):
        func = normalize(term.func)
        arg = normalize(term.arg)
        if isinstance(func, Lambda):
            return normalize(substitute(func.body, func.var, arg))
        return App(func, arg)
    elif isinstance(term, Lambda):
        return Lambda(term.var, term.var_type, normalize(term.body))
    elif isinstance(term, Pi):
        return Pi(term.var, term.var_type, normalize(term.body))
    else:
        return term

# === Nat Type and Constructors ===

Nat = Var("Nat")
Zero = Var("Zero")
Succ = Var("Succ")

# Example Type Environment for Natural Numbers
global_env = Env({
    'Nat': Type(),
    'Zero': Nat,
    'Succ': Pi('n', Nat, Nat)
})

# Example: plus : Nat -> Nat -> Nat
plus_type = Pi('n', Nat, Pi('m', Nat, Nat))
global_env['plus'] = plus_type

# Example: add_comm theorem skeleton
add_comm_type = Pi('n', Nat, Pi('m', Nat, App(App(Var('eq'), App(App(Var('plus'), Var('n')), Var('m'))), App(App(Var('plus'), Var('m')), Var('n')))))

global_env['eq'] = Pi('A', Type(), Pi('x', Var('A'), Pi('y', Var('A'), Type())))

def check_add_comm_type():
    print("Type of add_comm:", add_comm_type)
    t = type_check(global_env, add_comm_type)
    print("Type checking result:", t)

check_add_comm_type()

print("All syntax and type-checking routines loaded successfully.")

Type of add_comm: (n:Nat) -> (m:Nat) -> ((eq ((plus n) m)) ((plus m) n))


TypeError: Argument type mismatch: expected Type, got Nat